In [ ]:
!pip install git+https://github.com/m-bain/whisperx.git
!pip install torch
!pip install tensorflow[and-cuda]
!pip install tensorrt

In [ ]:
import whisperx
import gc
import os
import spacy
import torch
import copy
import pickle

/usr/local/lib/python3.10/dist-packages/pyannote/audio/core/io.py:43: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")
/usr/local/lib/python3.10/dist-packages/torch_audiomentations/utils/io.py:27: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")


In [ ]:
torch.cuda.is_available()

True

In [ ]:
# Фильтрация предложения по точкам
def filter_dot(result_old=False):
    if (type(result_old) == bool):
        print("result is not defind")

    result = copy.deepcopy(result_old['segments'])
    new_result = []


    for sentence in result:
        if len(new_result) == 0:
            new_result.append(sentence)
        else:
            if (new_result[-1]['text'][-1]) not in '.!?':
                new_result[-1]['end'] = sentence['end']
                new_result[-1]['text'] = new_result[-1]['text'] + sentence['text']
                new_result[-1]['words'] = new_result[-1]['words'] + sentence['words']
            else:
                new_result.append(sentence)

    result_old['segments'] = new_result
    return result_old

# Массив из предложений текста
def all_text_and_sentence_and_word(result=False):
    if (type(result) == bool):
        print("result is not defind")

    result = filter_dot(result)

    data_text = list()

    for sentence in result['segments']:
        data_text.append(sentence['text'])

    result['all_text'] = data_text

    return result


# Загрузка модели
def upload_model_whisperx(model="large"):
    device = "cuda"
    compute_type = "float16"
    model = whisperx.load_model(model, device, compute_type=compute_type, language="ru")
    return model

# Фильирация выявления предложения из текста
def stt(audio_file, model):
    if not(os.path.exists(audio_file)):
        print("file not exist")
        return False

    batch_size = 16
    device = "cuda"

    audio = whisperx.load_audio(audio_file)
    result = model.transcribe(audio, batch_size=batch_size)
    # result = model.transcribe(audio)

    model_a, metadata = whisperx.load_align_model(language_code=result["language"], device=device)
    result = whisperx.align(result["segments"], model_a, metadata, audio, device, return_char_alignments=False)

    gc.collect()
    torch.cuda.empty_cache()
    del model_a

    result = all_text_and_sentence_and_word(result)

    return result

In [ ]:
model = upload_model_whisperx()

INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.1.2. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/torch/whisperx-vad-segmentation.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.1.0. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.1.0+cu118. Bad things might happen unless you revert torch to 1.x.


In [ ]:
# Создание датасета из аудио файлов
data_set = list()
for i in range(1,7):
  print(f"start stt audio{i}.mp3")
  data_set.append(stt(f"data/audio{i}.mp3", model))
  print(f"end stt audio{i}.mp3 \n\n\n")

start stt audio1.mp3


Some weights of the model checkpoint at jonatasgrosman/wav2vec2-large-xlsr-53-russian were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_v', 'wav2vec2.encoder.pos_conv_embed.conv.weight_g']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at jonatasgrosman/wav2vec2-large-xlsr-53-russian and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
Y

end stt audio1.mp3 



start stt audio2.mp3


Some weights of the model checkpoint at jonatasgrosman/wav2vec2-large-xlsr-53-russian were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_v', 'wav2vec2.encoder.pos_conv_embed.conv.weight_g']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at jonatasgrosman/wav2vec2-large-xlsr-53-russian and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
Y

end stt audio2.mp3 



start stt audio3.mp3


...

end stt audio4.mp3 



start stt audio5.mp3


Some weights of the model checkpoint at jonatasgrosman/wav2vec2-large-xlsr-53-russian were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_v', 'wav2vec2.encoder.pos_conv_embed.conv.weight_g']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at jonatasgrosman/wav2vec2-large-xlsr-53-russian and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
Y

end stt audio5.mp3 



start stt audio6.mp3


Some weights of the model checkpoint at jonatasgrosman/wav2vec2-large-xlsr-53-russian were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_v', 'wav2vec2.encoder.pos_conv_embed.conv.weight_g']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at jonatasgrosman/wav2vec2-large-xlsr-53-russian and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
Y

end stt audio6.mp3 





In [ ]:
# Сохранение датасета в файл
with open('train_data.pkl', 'wb') as file:
    pickle.dump(data_set, file)

Для окрытия файла


```
with open(file_path, 'rb') as file:
    loaded_data = pickle.load(file)
```

